# Loan Limit Optimization — Documented Notebook
This notebook reproduces the cleaned analysis pipeline but adds inline explanations so a technical reviewer can follow the steps.
Cells are grouped: setup, constants, data loading, feature engineering, models, optimization, simulation, and outputs.
Keep the input file `loan_limit_increases.xlsx` in the same folder before running.

## Setup: imports and reproducibility
We import common data and ML libraries and fix random seeds so runs are repeatable.

In [2]:
# Standard library + numeric/data libs
import os
import random
from typing import Optional
import numpy as np
import pandas as pd
# Scikit-learn models and utilities
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

# Reproducibility: set an explicit seed for random and numpy to make results deterministic where possible
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# Note: some algorithms (e.g., multithreaded trees) may still introduce nondeterminism across platforms

## Constants and filenames
Keep these central so the notebook is easy to adjust for different simulation sizes or output names.

In [3]:
# Business and model constants (kept from the original script)
PROFIT_PER_INCREASE = 40  # profit recognized per successful increase (assumption)
MAX_INCREASES_PER_YEAR = 6  # operational or policy cap per customer
DISCOUNT_RATE = 0.19  # annual discount rate used for NPV calculations
ELIGIBILITY_THRESHOLD_DAYS = 60  # days since last loan required to be eligible

# Filenames used for input/output in the project
INPUT_FILE = 'loan_limit_increases.xlsx'  # expected Excel in working dir
OUT_RESULTS = 'loan_optimization_results.csv'
OUT_RECOMM = 'recommended_increases.csv'
OUT_SIM = 'simulation_results.csv'

# Small note: you can change these names if you want multiple experiment outputs

## Data loading
Load the Excel input and perform a small header normalization step to handle files where the first row contains human-readable headers.

In [4]:
def load_data(path: str) -> pd.DataFrame:
    """Load the dataset from Excel and coerce numeric columns.
    The function also detects when the first row contains header labels exported from other tools and corrects that.
    Returns: DataFrame with numeric conversions applied where possible.
    """
    if not os.path.exists(path):
        raise FileNotFoundError(f"Input file not found: {path}")
    # Read the sheet into a DataFrame; keep all rows, we'll detect header issues below
    df = pd.read_excel(path, skiprows=0)
    # Robustness: sometimes Excel exports put the true header on the first row. Detect and fix it
    if 'Customer ID' not in df.columns:
        first_row = df.iloc[0].astype(str).str.lower()
        if first_row.str.contains('customer id').any():
            # promote the first row to be the header and drop it from data rows
            df.columns = df.iloc[0]
            df = df.iloc[1:].reset_index(drop=True)
    # Coerce numeric columns where appropriate to avoid dtype surprises later
    for col in df.columns:
        if col != 'Customer ID':
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

## Feature engineering
We add simple derived features used by the uptake and default models. These are intentionally straightforward so the reasoning is clear.

In [5]:
def feature_engineering(df: pd.DataFrame) -> pd.DataFrame:
    # Work on a copy so we don't modify the caller's DataFrame in place
    df = df.copy()
    # Eligibility based on policy (days since last loan)
    df['Eligible'] = (df['Days Since Last Loan'] >= ELIGIBILITY_THRESHOLD_DAYS).astype(int)
    # Historical indicator: did the customer receive an increase in the prior year?
    df['Received_Increase'] = (df['No. of Increases in 2023'] > 0).astype(int)
    # Simple risk bucketing from on-time payment percent
    def assign_risk(payment_rate):
        if payment_rate >= 95: return 'Prime'
        if payment_rate >= 85: return 'Near-Prime'
        return 'Sub-Prime'
    df['Risk_Category'] = df['On-time Payments (%)'].apply(assign_risk)
    # Loan size buckets can help tree-based models capture scale effects
    df['Loan_Size_Category'] = pd.cut(df['Initial Loan ($)'], bins=[0,1500,3000,5000], labels=['Small','Medium','Large'])
    # A proxy credit score synthesized from available columns (for demo / modeling purposes)
    df['Credit_Score_Proxy'] = (
        df['On-time Payments (%)'] * 0.6 +
        (df['Days Since Last Loan'] / df['Days Since Last Loan'].max() * 100) * 0.2 +
        ((df['Initial Loan ($)'] / df['Initial Loan ($)'].max()) * 100) * 0.2
    )
    # Interaction features that might improve model signals
    df['Payment_Days_Interaction'] = df['On-time Payments (%)'] * df['Days Since Last Loan'] / 100
    df['Loan_Payment_Ratio'] = df['Initial Loan ($)'] / (df['On-time Payments (%)'] + 1)
    return df

## Uptake modeling (who accepts an increase)
We train a few candidate models and pick the best by AUC. The chosen model is used to predict an `Uptake_Probability` for every customer.

In [6]:
def build_uptake_models(df: pd.DataFrame):
    # Feature list used by the uptake models (kept concise)
    features = ['Initial Loan ($)','Days Since Last Loan','On-time Payments (%)','Credit_Score_Proxy','Payment_Days_Interaction','Loan_Payment_Ratio']
    X = df[features]
    y = df['Received_Increase']  # historical indicator used as the training target
    # Split for model selection; stratify keeps class balance in train/test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED, stratify=y)
    # Standardize only where models expect scaled inputs (logistic regression)
    scaler = StandardScaler()
    X_train_sc = scaler.fit_transform(X_train)
    X_test_sc = scaler.transform(X_test)
    # Fit three models (simple baseline + trees)
    lr = LogisticRegression(random_state=RANDOM_SEED, max_iter=1000)
    lr.fit(X_train_sc, y_train)
    rf = RandomForestClassifier(n_estimators=100, random_state=RANDOM_SEED, max_depth=10)
    rf.fit(X_train, y_train)  # trees don't need scaling
    gb = GradientBoostingClassifier(n_estimators=100, random_state=RANDOM_SEED, max_depth=5)
    gb.fit(X_train, y_train)
    models = {'Logistic Regression': (lr, X_test_sc), 'Random Forest': (rf, X_test), 'Gradient Boosting': (gb, X_test)}
    # Evaluate AUC on the test split to pick the best model
    best_name, best_auc, best_model = None, -1.0, None
    for name, (model, Xt) in models.items():
        try:
            proba = model.predict_proba(Xt)[:,1]
            auc = roc_auc_score(y_test, proba)
        except Exception:
            auc = 0.0
        if auc > best_auc:
            best_auc = auc
            best_name = name
            best_model = model
    # Use the best model to predict uptake probability for all rows
    if best_name == 'Logistic Regression':
        df['Uptake_Probability'] = lr.predict_proba(scaler.transform(df[features]))[:,1]
    else:
        df['Uptake_Probability'] = best_model.predict_proba(df[features])[:,1]
    return best_name, best_auc, df

## Default risk modeling
We create a simple heuristic default score and map it to a probability using a logistic function. This is intentionally interpretable rather than a black-box model.

In [ ]:
def build_default_model(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # Compose a risk score from observable signals (higher = worse)
    df['Default_Risk_Score'] = (
        (100 - df['On-time Payments (%)']) * 0.5 +
        (100 - df['Credit_Score_Proxy']) * 0.3 +
        (df['Initial Loan ($)'] / df['Initial Loan ($)'].max() * 100) * 0.2
    )
    # Map score to probability (sigmoid); this yields values in (0,1)
    df['Default_Probability'] = 1 / (1 + np.exp(-0.1 * (df['Default_Risk_Score'] - 50)))
    # Heuristic: previous increases slightly raise default risk
    df['Adjusted_Default_Probability'] = df['Default_Probability'] * (1 + 0.05 * df['No. of Increases in 2023'])
    df['Adjusted_Default_Probability'] = df['Adjusted_Default_Probability'].clip(0, 0.95)  # cap at 95%
    return df

def calculate_expected_value(row):
    # Expected value (per potential increase) = profit * P(uptake) * (1 - P(default)) - expected loss from defaults
    uptake_prob = row['Uptake_Probability']
    default_prob = row['Adjusted_Default_Probability']
    expected_profit = PROFIT_PER_INCREASE * uptake_prob * (1 - default_prob)
    expected_loss = row['Initial Loan ($)'] * 0.5 * uptake_prob * default_prob
    return expected_profit - expected_loss

## Optimization heuristic
A greedy approach: rank eligible customers by a risk-adjusted expected value and recommend a small number of increases per customer while respecting high-risk limits and an optional capital constraint.

In [ ]:
def optimize_loan_increases(df_input, max_high_risk_pct=0.25, capital_constraint=None):
    # Work only on eligible customers
    eligible = df_input[df_input['Eligible'] == 1].copy()
    # Sort by a risk-adjusted score so top rows are prioritized
    eligible = eligible.sort_values('Risk_Adjusted_Score', ascending=False)
    eligible['Recommended_Increases'] = 0
    eligible['Total_Expected_Value'] = 0.0
    total_value = total_exposure = 0.0
    high_risk_count = total_approvals = 0
    # Iterate and greedily allocate increases when expected value is positive
    for idx, row in eligible.iterrows():
        is_high_risk = row['Risk_Category'] == 'Sub-Prime'
        # enforce a maximum percentage of approvals that can be high-risk
        if is_high_risk and high_risk_count >= len(eligible) * max_high_risk_pct:
            continue
        if row['Expected_Value'] <= 0:  # skip negative expectation rows
            continue
        # choose a reasonable number of increases based on predicted uptake
        optimal_increases = min(MAX_INCREASES_PER_YEAR, int(row['Uptake_Probability'] * MAX_INCREASES_PER_YEAR) + 1)
        if capital_constraint:
            projected_exposure = row['Initial Loan ($)'] * optimal_increases * 0.5
            if total_exposure + projected_exposure > capital_constraint:
                continue
        eligible.at[idx, 'Recommended_Increases'] = optimal_increases
        eligible.at[idx, 'Total_Expected_Value'] = float(row['Expected_Value']) * optimal_increases
        total_value += eligible.at[idx, 'Total_Expected_Value']
        total_exposure += row['Initial Loan ($)'] * optimal_increases * 0.5
        if is_high_risk:
            high_risk_count += 1
        total_approvals += 1
    return {'eligible_df': eligible, 'total_expected_value': total_value, 'total_approvals': total_approvals, 'total_exposure': total_exposure, 'high_risk_count': high_risk_count, 'high_risk_pct': high_risk_count / total_approvals if total_approvals > 0 else 0}

## Monte Carlo simulation (lifecycle per customer)
Simulate transitions between risk states and whether increases are accepted and whether defaults occur. This estimates the distribution of net value per customer.

In [ ]:
def simulate_loan_lifecycle(customer_row, n_simulations=100, time_periods=4):
    # We'll collect a small results table per simulation run for each customer
    results = []
    # Transition probabilities between risk buckets (rows sum to 1)
    transition_matrix = np.array([[0.85,0.12,0.03],[0.15,0.7,0.15],[0.05,0.25,0.7]])
    risk_states = ['Prime','Near-Prime','Sub-Prime']
    for sim in range(n_simulations):
        total_profit = total_losses = 0.0
        defaults = increases = 0
        # start from the customer's observed risk category
        current_risk_state = customer_row['Risk_Category']
        for quarter in range(time_periods):
            if quarter > 0:
                # sample next state using the transition matrix
                state_idx = {'Prime':0,'Near-Prime':1,'Sub-Prime':2}[current_risk_state]
                probs = transition_matrix[state_idx]
                next_state_idx = np.random.choice(3, p=probs)
                current_risk_state = risk_states[next_state_idx]
            # risk multiplier increases the default probability for worse buckets
            risk_multiplier = {'Prime':0.8,'Near-Prime':1.0,'Sub-Prime':1.3}[current_risk_state]
            adjusted_default_prob = min(customer_row['Default_Probability'] * risk_multiplier, 0.95)
            # check whether customer accepts the product this quarter
            accepts = np.random.random() < customer_row['Uptake_Probability']
            if accepts:
                increases += 1
                if np.random.random() < adjusted_default_prob:
                    defaults += 1
                    # assume a 50% loss given default on outstanding amount
                    total_losses += customer_row['Initial Loan ($)'] * 0.5
                else:
                    total_profit += PROFIT_PER_INCREASE
        results.append({'simulation': sim, 'total_profit': total_profit, 'total_losses': total_losses, 'net_value': total_profit - total_losses, 'defaults': defaults, 'increases_granted': increases})
    return pd.DataFrame(results)

def calculate_npv(cash_flows, discount_rate=DISCOUNT_RATE):
    # Present-value calculation with quarterly discounting approximation
    npv = 0.0
    for t, cf in enumerate(cash_flows):
        npv += cf / ((1 + discount_rate) ** (t / 4))
    return npv

## Run the pipeline (example)
Adjust `SIM_CUSTOMER_COUNT`, `N_SIMULATIONS_PER_CUSTOMER`, and `SIM_QUARTERS` below before executing this cell to change computational footprint.

In [ ]:
# Example runtime parameters
SIM_CUSTOMER_COUNT = 1000  # number of unique customers to sample for MC
N_SIMULATIONS_PER_CUSTOMER = 100  # Monte Carlo runs per sampled customer
SIM_QUARTERS = 4  # number of quarters to simulate

# Load and prepare data
df = load_data(INPUT_FILE)
print('Dataset Shape:', df.shape)
df = feature_engineering(df)
print('Eligible Customers (>=60 days):', int(df['Eligible'].sum()))

# Fit uptake models and predict probabilities
best_name, best_auc, df = build_uptake_models(df)
print('Best uptake model:', best_name, 'AUC=', round(best_auc,4))
# Build default model and adjusted default probabilities
df = build_default_model(df)

# Compute expected value per potential increase and a risk-adjusted prioritization score
df['Expected_Value'] = df.apply(calculate_expected_value, axis=1)
df['Risk_Adjusted_Score'] = df['Expected_Value'] * (1 - df['Adjusted_Default_Probability']) * df['Uptake_Probability']

# Run greedy allocation heuristic
optimization_results = optimize_loan_increases(df, max_high_risk_pct=0.25)
print('Approved for Increases:', optimization_results['total_approvals'])
print('Total Expected Value: $', round(optimization_results['total_expected_value'],2))

# Monte Carlo sampling: sample eligible customers (with replacement if sample > available)
eligible_customers = df[df['Eligible'] == 1]
available = len(eligible_customers)
if SIM_CUSTOMER_COUNT <= available:
    sample_customers = eligible_customers.sample(n=SIM_CUSTOMER_COUNT, random_state=RANDOM_SEED)
else:
    sample_customers = eligible_customers.sample(n=SIM_CUSTOMER_COUNT, replace=True, random_state=RANDOM_SEED)

# Run simulations for each sampled customer and collect results
sim_list = []
for _, row in sample_customers.iterrows():
    sim_df = simulate_loan_lifecycle(row, n_simulations=N_SIMULATIONS_PER_CUSTOMER, time_periods=SIM_QUARTERS)
    sim_df['customer_id'] = row['Customer ID']
    sim_df['risk_category'] = row['Risk_Category']
    sim_list.append(sim_df)
all_simulations = pd.concat(sim_list, ignore_index=True)

print('Total Simulation Runs:', SIM_CUSTOMER_COUNT * N_SIMULATIONS_PER_CUSTOMER)
print('Total Individual Decisions:', SIM_CUSTOMER_COUNT * N_SIMULATIONS_PER_CUSTOMER * SIM_QUARTERS)
print('Simulation sample mean net value:', round(all_simulations.groupby('customer_id')['net_value'].mean().mean(),2))

# Persist outputs (CSV files) so we can use the cleaned visualization script or create figures in another notebook
df.to_csv(OUT_RESULTS, index=False)
optimization_results['eligible_df'][optimization_results['eligible_df']['Recommended_Increases'] > 0][['Customer ID','Risk_Category','On-time Payments (%)','Initial Loan ($)','Uptake_Probability','Default_Probability','Expected_Value','Recommended_Increases','Total_Expected_Value']].to_csv(OUT_RECOMM, index=False)
all_simulations.to_csv(OUT_SIM, index=False)
print('Saved outputs:', OUT_RESULTS, OUT_RECOMM, OUT_SIM)

# End of pipeline run cell

## Next steps and tips
- If you'd like, I can: convert the visualization script into notebook cells, run the notebook end-to-end here to validate outputs, or replace the original notebook with this documented version.
- For large simulation footprints (e.g., millions of runs) consider running on a machine with more CPU/memory or reduce `N_SIMULATIONS_PER_CUSTOMER`.